# Q1

In [1]:
from scipy.io import loadmat
annots = loadmat('P1.mat')

In [2]:
XX = annots['XX']

In [3]:
XX = XX.T
XX.shape

(20, 300, 6)

for our 6x300x20 tensor , we have 6 nodes , and each node has 20 vectors of 300 observations.
we have to compute the directed informations for all pairs of X-Y nodes , with Z the remaining nodes.
t is in (0,20)
 and n is in (0,300)

The steps for our strategy will be as follows:
- for every possible pair of nodes i, j
    - we will have the node we will start from in s_x the target node in s_y and gather the remaining nodes in s_Z
    - next we will construct x_t and y_t, x_t will be the start vector and y_t the target vector
    - z_t will be a matrice covering all remaining vectors.
    - x_t,y_t and z_t are the vectors of the time series of the corresponding set of nodes s_x, s_y and s_z
    - we will concatenate the necessary vectors to create the matrices (m1, m2, m3, m4) for computing the covariance as explained by the hint, this will be our metric for directed information.
    - Once we have the matrices m1, m2, m3, m4 we will compute the determinant of the covariance of these matrices and use it in the formula we have in the hint to compare it with our threshold of 0.6 and infer if there is an edge from node i to j.

In [4]:
import numpy as np

In [5]:
I = np.zeros((6,6))

In [11]:
import math
from scipy.linalg import det
from numpy import cov
import numpy as np

all_nodes ={0, 1, 2, 3, 4, 5}

T = 20

for i in all_nodes:
    for j in all_nodes:
        if i == j:
            continue
        else:
            s_x = {i}
            s_y = {j}
            s_z = all_nodes - s_x - s_y
            x_t = XX[:,:,i]  #company i
            y_t = XX[:,:,j]  #company j

            z_t = [] #remaining companies
            for k in s_z:
                z_t.append(XX[:,:,k])
                
            z_t = np.asarray(z_t)
            
            k = 0
            for t in np.arange(2,T):
                
                m1 = np.concatenate((y_t[:t,:],z_t[:,:t-1,:].reshape(-1,300)))
                m2 = np.concatenate( (x_t[:t-1,:] , np.concatenate((y_t[:t-1,:],z_t[:,:t-1,:].reshape(-1,300)))) )
                m3 = np.concatenate((y_t[:t-1,:],z_t[:,:t-1,:].reshape(-1,300)))
                m4 = np.concatenate( (x_t[:t-1,:] , np.concatenate((y_t[:t,:],z_t[:,:t-1,:].reshape(-1,300)))) )
                
                c1 = cov(m1)
                d1 = det(c1)
                
                c2 = cov(m2)
                d2 = det(c2)
               
                c3 = cov(m3)
                d3 = det(c3)
                
                c4 = cov(m4)                
                d4 = det(c4)
                
                k += np.log( (d1 * d2) / (d3 * d4) )
            
            DI = 0.5 * k
            if DI < 0.6:
                k = 0
            else:
                k = 1
                
        I[i][j] = k
        

In [12]:
I

array([[0., 1., 1., 0., 0., 1.],
       [0., 0., 1., 1., 0., 0.],
       [1., 0., 0., 1., 0., 1.],
       [0., 1., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 1., 1., 0.]])

# Q2

For this question we will use the least squares function provided in the linalg library of numpy to estimate the coefficient matrice A

In [8]:
data = XX[:]

In [9]:
X = np.asarray(XX[:,:,1:20].reshape(6,-1)[::-1].T)
Y = np.asarray(XX[:,:,0:19].reshape(6,-1)[::-1].T)

In [10]:
a = np.linalg.lstsq(X, Y)
a

LinAlgError: Incompatible dimensions

In [ ]:
A = a[0]
A

In [ ]:
A.shape